In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

##13.1 데이터 API

In [3]:
X = tf.range(10)
_dataset = tf.data.Dataset.from_tensor_slices(X)
_dataset

<TensorSliceDataset shapes: (), types: tf.int32>

```python
tf.data.Dataset.range(10)
```

In [4]:
for item in _dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


###13.1.1 연쇄 변환

#### repeat & batch

In [5]:
dataset = _dataset.repeat(3).batch(7, drop_remainder=False) # inplace = False
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [6]:
dataset = _dataset.repeat(3).batch(4, drop_remainder=True).batch(2) # inplace = False
for item in dataset:
    print(item)

tf.Tensor(
[[0 1 2 3]
 [4 5 6 7]], shape=(2, 4), dtype=int32)
tf.Tensor(
[[8 9 0 1]
 [2 3 4 5]], shape=(2, 4), dtype=int32)
tf.Tensor(
[[6 7 8 9]
 [0 1 2 3]], shape=(2, 4), dtype=int32)
tf.Tensor([[4 5 6 7]], shape=(1, 4), dtype=int32)


#### map

In [7]:
square_dataset = _dataset.map(lambda x : x ** 2)
for sq in square_dataset:
    print(sq)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(36, shape=(), dtype=int32)
tf.Tensor(49, shape=(), dtype=int32)
tf.Tensor(64, shape=(), dtype=int32)
tf.Tensor(81, shape=(), dtype=int32)


####filter

In [8]:
for ft in _dataset.filter(lambda x : x < 5):
    print(ft)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


####take

In [9]:
for tk in _dataset.take(3):
    print(tk)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)


###13.1.2 데이터 셔플링

####shuffle

In [10]:
dataset = _dataset.repeat(3)
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int32)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int32)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int32)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int32)
tf.Tensor([3 6], shape=(2,), dtype=int32)


#### csv 파일 생성

In [11]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

In [12]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

In [13]:
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

#### 여러 파일에서 한 줄씩 번갈아 읽기

In [14]:
train_filepaths

['datasets/housing/my_train_00.csv',
 'datasets/housing/my_train_01.csv',
 'datasets/housing/my_train_02.csv',
 'datasets/housing/my_train_03.csv',
 'datasets/housing/my_train_04.csv',
 'datasets/housing/my_train_05.csv',
 'datasets/housing/my_train_06.csv',
 'datasets/housing/my_train_07.csv',
 'datasets/housing/my_train_08.csv',
 'datasets/housing/my_train_09.csv',
 'datasets/housing/my_train_10.csv',
 'datasets/housing/my_train_11.csv',
 'datasets/housing/my_train_12.csv',
 'datasets/housing/my_train_13.csv',
 'datasets/housing/my_train_14.csv',
 'datasets/housing/my_train_15.csv',
 'datasets/housing/my_train_16.csv',
 'datasets/housing/my_train_17.csv',
 'datasets/housing/my_train_18.csv',
 'datasets/housing/my_train_19.csv']

#### list_files

In [15]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)
for path in filepath_dataset:
    print(path)

tf.Tensor(b'datasets/housing/my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_14.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_02.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_13.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_15.csv', shape=(), dtype=string)
tf.Ten

#### interleave

#### TextLineDataset

In [16]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath),#.skip(5),
    cycle_length=n_readers)
for line in dataset.take(10):
    print(line.numpy())

b'MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue'
b'MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue'
b'MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue'
b'MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue'
b'MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue'
b'4.5909,16.0,5.475877192982456,1.0964912280701755,1357.0,2.9758771929824563,33.63,-117.71,2.418'
b'2.4792,24.0,3.4547038327526134,1.1341463414634145,2251.0,3.921602787456446,34.18,-118.38,2.0'
b'4.2708,45.0,5.121387283236994,0.953757225433526,492.0,2.8439306358381504,37.48,-122.19,2.67'
b'2.1856,41.0,3.7189873417721517,1.0658227848101265,803.0,2.0329113924050635,32.76,-117.12,1.205'
b'4.1812,52.0,5.701388888888889,0.9965277777777778,692.0,2.4027777777777777,33.73,-118.31,3.215'


### 13.1.3 데이터 전처리

In [17]:
import pandas as pd

In [18]:
file_path = '/content/sample_data/california_housing_train.csv'
housing = pd.read_csv(file_path)
X_train = housing.iloc[:, :8]
y_train = housing.iloc[:, -1]

In [19]:
X_mean, X_std = X_train.mean().values, X_train.std().values
n_inputs = 8

####decode_csv

####stack

In [20]:
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x-X_mean) / X_std, y

In [21]:
defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
print(defs)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, <tf.Tensor: shape=(0,), dtype=float32, numpy=array([], dtype=float32)>]


In [22]:
f = open(file_path)
lines = f.readlines()

In [23]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(5),
    cycle_length=n_readers)
for line in dataset.take(5):
    print(preprocess(line.numpy()))

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([ 62.15784   ,  -0.29252487,  -1.8785055 ,  -1.212307  ,
         0.28846815,  -1.2425393 ,  -1.2154919 , -63.864563  ],
      dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.896], dtype=float32)>)
(<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([ 60.95016  ,  -3.0997527,  -1.86399  ,  -1.2121627,   2.5565612,
        -1.2423905,  -1.2125791, -65.15376  ], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.083], dtype=float32)>)
(<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([ 62.767467 , -10.585693 ,  -1.7748046,  -1.2122576,   2.988353 ,
        -1.2427121,  -1.2139834, -64.10039  ], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.183], dtype=float32)>)
(<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([ 67.10776   ,   2.9825742 ,  -1.5910823 ,  -1.2122772 ,
         0.21966618,  -1.2429751 ,  -1.2149458 , -64.131836  ],
      dtype=float32)>, <tf.Tenso

### 13.1.4 데이터 적재와 전처리를 합치기

####prefetch

In [24]:
def csv_reader_dataset(filepath, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    filepath_dataset = tf.data.Dataset.list_files(filepath).repeat(repeat)
    interleave_dataset = filepath_dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads
    )
    dataset = interleave_dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)

In [25]:
dataset = csv_reader_dataset(file_path)
for item in dataset.take(1):
    x, y = item
    print(x.shape, y.shape)

(32, 8) (32, 1)


#### concatenate

####zip

###13.1.6 tf.keras와 데이터셋 사용하기

In [26]:
train_set = csv_reader_dataset(train_filepaths)

##13.2 TFRecord 포맷

####TFRecordWriter

In [27]:
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"This is the first record")
    f.write(b"And this is the second record")

####TFRecordDataset

In [28]:
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


##13.3 입력 특성 전처리

####Lambda

```python
model = keras.models.Sequential([
                                    keras.layers.Lambda(lambda input: (inputs - means) / (stds + eps)),
                                 [...]
])
```

#### BatchNormalization

In [29]:
class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())

In [30]:
std_layer = Standardization()
std_layer.adapt(np.array(X_train))
std_layer.means_, std_layer.stds_

(array([[-119.56210824,   35.62522471,   28.58935294, 2643.66441176,
          539.41082353, 1429.57394118,  501.22194118,    3.8835781 ]]),
 array([[2.00510743e+00, 2.13727693e+00, 1.25865668e+01, 2.17988295e+03,
         4.21487054e+02, 1.14781920e+03, 3.84509531e+02, 1.90810040e+00]]))

###13.3.1 원-핫 벡터를 사용해 범주형 특성 인코딩하기

In [31]:
X_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250
...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797


####KeyValueTensorInitializer

In [32]:
vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)

In [35]:
tf.lookup.KeyValueTensorInitializer?

####StaticVocabularyTable

In [34]:
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

In [42]:
table.lookup("INLAND")

AttributeError: ignored